In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%run init.py

Created snorkel session from  postgres:///snorkel25similar


/home/antonis/anaconda2/envs/snorkel27/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
train = session.query(REGULATOR).filter(REGULATOR.split == 0).order_by(REGULATOR.id).all()
dev = session.query(REGULATOR).filter(REGULATOR.split == 1).order_by(REGULATOR.id).all()
test = session.query(REGULATOR).filter(REGULATOR.split == 2).order_by(REGULATOR.id).all()
unlab = session.query(REGULATOR).filter(REGULATOR.split == 3).order_by(REGULATOR.id).all()

print('Training set:\t{0} candidates'.format(len(train)))
print('Dev set:\t{0} candidates'.format(len(dev)))
print('Test set:\t{0} candidates'.format(len(test)))
print('Unlab set:\t{0} candidates'.format(len(unlab)))

Training set:	13007 candidates
Dev set:	3242 candidates
Test set:	8221 candidates
Unlab set:	23509 candidates


In [3]:
from snorkel.annotations import load_gold_labels

L_gold_train  = load_gold_labels(session, annotator_name='gold', split=0)
L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

In [4]:
import numpy as np

In [34]:
unlab_marginals = load_marginals(session, split=3) # safe - loads them after ordering by candidate id

In [23]:
from snorkel.annotations import load_marginals
train_labels = np.array([0 if train[x].gold_labels[0].value ==-1 else 1 for x in range(len(train))]) # load train labels to mix in LSTM
dev_labels = np.array([0 if dev[x].gold_labels[0].value ==-1 else 1 for x in range(len(dev))]) # load train labels to mix in LSTM
test_labels = np.array([0 if test[x].gold_labels[0].value ==-1 else 1 for x in range(len(test))]) # load train labels to mix in LSTM

unlab_marginals = load_marginals(session, split=3) # safe - snorkel orders them by Candidate id


In [13]:
# with open("L_train_us_mod8", 'rb') as f:
#     unlab, unlab_marginals = pickle.load(f)

In [6]:
# unlab

In [8]:
# len(unlab_marginals)

In [8]:
np.__version__

'1.14.5'

In [9]:
from snorkel.learning import reRNN

/home/antonis/anaconda2/envs/snorkel27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [10]:
from MLutils import balance_candidates

In [49]:
# unlab_us,unlab_marginals_us = balance_candidates(unlab,unlab_marginals)

# # TODO: balance candidates also in training set (change train_unlab_marginals below + train -> train_us in lstm.train)
train_us, train_labels_us = balance_candidates(train,train_labels)

train_unlab_marginals = np.append(train_labels_us, unlab_marginals)

In [13]:
# import matplotlib.pyplot as plt
# plt.hist(unlab_marginals, bins=20, range=[0,1])
# plt.title('Histogram of probabilistic labels (unlab set)')
# plt.show()

# import matplotlib.pyplot as plt
# plt.hist(train_labels_us, bins=20, range=[0,1])
# plt.title('Histogram of GS labels')
# plt.show()

# # plt.hist(unlab_marginals_us, bins=20, range=[0,1])
# # plt.title('Histogram of probabilistic labels (unlab set, undersampled)')
# # plt.show()

# # plt.hist(train_labels_us, bins=20)
# # plt.title('Histogram of labels (train set)')
# # plt.show()

# plt.hist(train_unlab_marginals, bins=20)
# plt.title('Histogram of probabilistic labels (train+unlab set)')
# plt.show()


** train on train_set (to see if all ok or theres a mess-up in labels) **

In [14]:
len(train)

13007

In [15]:
train_labels.shape

(13007,)

In [16]:
L_gold_dev

<3242x1 sparse matrix of type '<type 'numpy.int64'>'
	with 3242 stored elements in Compressed Sparse Row format>

In [18]:
train_labels

array([1, 0, 0, ..., 0, 0, 1])

In [22]:
train_kwargs = {
    'lr':         0.01,
    'dim':        100,
    'n_epochs':   10,
    'dropout':    0.25,
    'rebalance':  True,
    'print_freq': 1,
    'batch_size': 64
}

lstm = reRNN(seed=1701, deterministic=True)
lstm.train(train, train_labels, X_dev=dev, Y_dev=L_gold_dev, **train_kwargs)


/home/antonis/anaconda2/envs/snorkel27/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[reRNN] Training model
[reRNN] n_train=6576  #epochs=10  batch size=64
[reRNN] Epoch 0 (17.63s)	Average loss=0.595465	Dev F1=47.78
[reRNN] Epoch 1 (37.58s)	Average loss=0.384189	Dev F1=48.76
[reRNN] Epoch 2 (57.80s)	Average loss=0.285590	Dev F1=52.87
[reRNN] Epoch 3 (77.82s)	Average loss=0.218953	Dev F1=50.23
[reRNN] Epoch 4 (97.97s)	Average loss=0.171683	Dev F1=48.47
[reRNN] Epoch 5 (117.89s)	Average loss=0.143844	Dev F1=49.37
[reRNN] Epoch 6 (138.26s)	Average loss=0.110207	Dev F1=54.70
[reRNN] Epoch 7 (158.44s)	Average loss=0.087948	Dev F1=55.31
[reRNN] Epoch 8 (178.71s)	Average loss=0.075223	Dev F1=54.76
[reRNN] Model saved as <reRNN>
[reRNN] Epoch 9 (199.01s)	Average loss=0.069361	Dev F1=53.31
[reRNN] Training done (201.97s)
INFO:tensorflow:Restoring parameters from checkpoints/reRNN/reRNN-8
[reRNN] Loaded model <reRNN>


In [24]:
lstm.score(dev,L_gold_dev)

(0.516209476309227, 0.5830985915492958, 0.5476190476190477)

In [25]:
lstm.score(dev,dev_labels)

(0.516209476309227, 0.5830985915492958, 0.5476190476190477)

In [28]:
lstm.score(test,L_gold_test,batch_size=32)

(0.5305052430886559, 0.5609879032258065, 0.5453209211170995)

In [29]:
lstm.score(test,test_labels,batch_size=64)

(0.5305052430886559, 0.5609879032258065, 0.5453209211170995)

** train on unlab **

In [38]:
train_kwargs = {
    'lr':         0.01,
    'dim':        100,
    'n_epochs':   35,
    'dropout':    0.25,
    'rebalance':  True,
    'print_freq': 1,
    'batch_size': 64
}

lstm = reRNN(seed=1701, deterministic=True)
lstm.train(unlab, unlab_marginals, X_dev=dev, Y_dev=L_gold_dev, **train_kwargs)

/home/antonis/anaconda2/envs/snorkel27/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[reRNN] Training model
[reRNN] n_train=10786  #epochs=35  batch size=64
[reRNN] Epoch 0 (29.66s)	Average loss=0.605242	Dev F1=39.39
[reRNN] Epoch 1 (62.07s)	Average loss=0.441300	Dev F1=36.97
[reRNN] Epoch 2 (94.42s)	Average loss=0.358777	Dev F1=37.20
[reRNN] Epoch 3 (126.68s)	Average loss=0.326538	Dev F1=38.69
[reRNN] Epoch 4 (158.80s)	Average loss=0.309259	Dev F1=40.40
[reRNN] Epoch 5 (191.24s)	Average loss=0.295640	Dev F1=39.77
[reRNN] Epoch 6 (223.55s)	Average loss=0.286219	Dev F1=38.31
[reRNN] Epoch 7 (255.60s)	Average loss=0.280151	Dev F1=39.34
[reRNN] Epoch 8 (287.53s)	Average loss=0.272821	Dev F1=41.00
[reRNN] Epoch 9 (319.84s)	Average loss=0.268550	Dev F1=39.86
[reRNN] Epoch 10 (351.62s)	Average loss=0.266627	Dev F1=40.16
[reRNN] Epoch 11 (383.94s)	Average loss=0.263585	Dev F1=41.28
[reRNN] Epoch 12 (416.03s)	Average loss=0.258636	Dev F1=41.27
[reRNN] Epoch 13 (448.12s)	Average loss=0.255536	Dev F1=40.86
[reRNN] Epoch 14 (480.49s)	Average loss=0.253268	Dev F1=41.30
[reRNN] Epo

In [39]:
lstm.score(dev,L_gold_dev)

(0.3205456095481671, 0.5295774647887324, 0.3993627190653213)

In [40]:
lstm.score(dev,dev_labels)

(0.3205456095481671, 0.5295774647887324, 0.3993627190653213)

In [41]:
lstm.score(test,L_gold_test,batch_size=32)

(0.3261455525606469, 0.5488911290322581, 0.40916776253992115)

In [42]:
lstm.score(test,test_labels,batch_size=1024)

(0.3261455525606469, 0.5488911290322581, 0.40916776253992115)

In [43]:
tp, fp, tn, fn = lstm.error_analysis(session, dev, L_gold_dev)

Scores (Un-adjusted)
Pos. class accuracy: 0.53
Neg. class accuracy: 0.685
Precision            0.321
Recall               0.53
F1                   0.399
----------------------------------------
TP: 376 | FP: 797 | TN: 1735 | FN: 334



In [45]:
tp, fp, tn, fn = lstm.error_analysis(session, test, L_gold_test,batch_size=1024)

Scores (Un-adjusted)
Pos. class accuracy: 0.549
Neg. class accuracy: 0.639
Precision            0.326
Recall               0.549
F1                   0.409
----------------------------------------
TP: 1089 | FP: 2250 | TN: 3987 | FN: 895



** train on GS + unlab **

In [ ]:
train_kwargs = {
    'lr':         0.01,
    'dim':        100,
    'n_epochs':   25,
    'dropout':    0.25,
    'rebalance':  True,
    'print_freq': 1,
    'batch_size': 64
}

lstm = reRNN(seed=1701, deterministic=True)
lstm.train(train+unlab, 
           np.concatenate([train_labels ,unlab_marginals]),
           X_dev=dev, Y_dev=L_gold_dev, **train_kwargs)


/home/antonis/anaconda2/envs/snorkel27/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[reRNN] Training model
[reRNN] n_train=17362  #epochs=25  batch size=64
[reRNN] Epoch 0 (48.29s)	Average loss=0.571786	Dev F1=46.35
[reRNN] Epoch 1 (99.32s)	Average loss=0.402291	Dev F1=48.62


In [ ]:
tp, fp, tn, fn = lstm.error_analysis(session, dev, L_gold_dev)

In [ ]:
tp, fp, tn, fn = lstm.error_analysis(session, test, L_gold_test, batch_size = 1024)

# Check actual end-model score

In [13]:
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_test

<8335x1 sparse matrix of type '<type 'numpy.int64'>'
	with 8335 stored elements in Compressed Sparse Row format>

In [28]:
tp, fp, tn, fn = lstm.error_analysis(session, test, L_gold_test)

Scores (Un-adjusted)
Pos. class accuracy: 0.608
Neg. class accuracy: 0.54
Precision            0.293
Recall               0.608
F1                   0.395
----------------------------------------
TP: 1208 | FP: 2918 | TN: 3430 | FN: 779

